<a href="https://colab.research.google.com/github/Khush0714/Data_science_Lab_SE_74/blob/main/exp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #Demonstrate Breadth First Search and Depth First Search

In [1]:
graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['F'],
    'F': []
}

In [2]:
from collections import deque

def bfs(graph, start_node):
    visited = set()
    queue = deque([start_node])
    traversal_order = []

    visited.add(start_node)

    while queue:
        node = queue.popleft()
        traversal_order.append(node)

        for neighbor in graph[node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    return traversal_order

In [3]:
def dfs(graph, start_node):
    visited = set()
    stack = [start_node]
    traversal_order = []

    while stack:
        node = stack.pop()
        if node not in visited:
            visited.add(node)
            traversal_order.append(node)

            # Add neighbors to stack in reverse order to explore in alphabetical order (or similar consistent order)
            for neighbor in sorted(graph[node], reverse=True):
                if neighbor not in visited:
                    stack.append(neighbor)
    return traversal_order

# Recursive DFS alternative
def dfs_recursive(graph, start_node, visited=None, traversal_order=None):
    if visited is None:
        visited = set()
    if traversal_order is None:
        traversal_order = []

    visited.add(start_node)
    traversal_order.append(start_node)

    for neighbor in graph[start_node]:
        if neighbor not in visited:
            dfs_recursive(graph, neighbor, visited, traversal_order)
    return traversal_order

In [6]:
print(f"BFS Traversal (starting from A): {bfs(graph, 'A')}")
print(f"DFS Traversal (iterative, starting from A): {dfs(graph, 'A')}")
print(f"DFS Traversal (recursive, starting from A): {dfs_recursive(graph, 'A')}")

BFS Traversal (starting from A): ['A', 'B', 'C', 'D', 'E', 'F']
DFS Traversal (iterative, starting from A): ['A', 'B', 'D', 'E', 'F', 'C']
DFS Traversal (recursive, starting from A): ['A', 'B', 'D', 'E', 'F', 'C']


In [ ]:
#Implementation of A* algorithm.

In [ ]:
import heapq

def heuristic(a, b):
    """Manhattan distance heuristic"""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star(grid, start, end):
    # grid dimensions
    rows, cols = len(grid), len(grid[0])

    # Check if start or end are obstacles
    if grid[start[0]][start[1]] == 1 or grid[end[0]][end[1]] == 1:
        return None # Path not possible if start/end is an obstacle

    # Priority queue: (f_cost, g_cost, node)
    # f_cost is g_cost + h_cost, used for sorting
    # g_cost is actual cost from start to current node
    open_set = [(0, 0, start)] # (f_cost, g_cost, node)

    # Dictionary to store the g_cost of reaching each node
    g_costs = {start: 0}

    # Dictionary to reconstruct the path
    came_from = {}

    # Possible movements (up, down, left, right)
    movements = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    while open_set:
        f_cost, g_cost, current_node = heapq.heappop(open_set)

        if current_node == end:
            # Reconstruct path
            path = []
            while current_node in came_from:
                path.append(current_node)
                current_node = came_from[current_node]
            path.append(start)
            return path[::-1] # Return reversed path

        for dx, dy in movements:
            neighbor = (current_node[0] + dx, current_node[1] + dy)

            # Check if neighbor is within grid boundaries
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols:
                # Check if neighbor is not an obstacle (value 1)
                if grid[neighbor[0]][neighbor[1]] == 0:
                    # Cost to reach neighbor from start through current_node
                    tentative_g_cost = g_cost + 1 # Assuming cost of 1 for each step

                    if neighbor not in g_costs or tentative_g_cost < g_costs[neighbor]:
                        g_costs[neighbor] = tentative_g_cost
                        f_cost_neighbor = tentative_g_cost + heuristic(neighbor, end)
                        heapq.heappush(open_set, (f_cost_neighbor, tentative_g_cost, neighbor))
                        came_from[neighbor] = current_node

    return None # No path found

In [ ]:
# Define a sample grid (0 = traversable, 1 = obstacle)
grid = [
    [0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

start_node = (0, 0) # Top-left corner
end_node = (4, 4)   # Bottom-right corner

print("Grid:")
for row in grid:
    print(row)
print(f"\nStart: {start_node}")
print(f"End: {end_node}")

path = a_star(grid, start_node, end_node)

if path:
    print("\nPath found:")
    for r, c in path:
        print(f"({r}, {c})", end=" -> ")
    print("End")

    # Visualize the path on the grid
    path_grid = [row[:] for row in grid] # Create a copy of the grid
    for r, c in path:
        if (r, c) == start_node:
            path_grid[r][c] = 'S' # Start
        elif (r, c) == end_node:
            path_grid[r][c] = 'E' # End
        else:
            path_grid[r][c] = '*' # Path

    print("\nPath Visualization:")
    for r_idx, row in enumerate(path_grid):
        for c_idx, val in enumerate(row):
            if (r_idx, c_idx) == start_node:
                print(f" {val} ", end="")
            elif (r_idx, c_idx) == end_node:
                print(f" {val} ", end="")
            elif grid[r_idx][c_idx] == 1:
                print(" # ", end="") # Obstacle
            elif val == '*':
                print(" * ", end="") # Path
            else:
                print(" . ", end="") # Empty space
        print()

else:
    print("\nNo path found!")

In [8]:
#Implement the basic Minimax algorithm for two-player deterministic games.

In [9]:
def minimax(node_index, depth, maximizing_player, scores, tree_depth):
    # Base case: If max depth is reached or it's a leaf node
    if depth == tree_depth:
        return scores[node_index]

    # If it's a maximizing player's turn
    if maximizing_player:
        best_value = -float('inf') # Initialize with negative infinity
        # Consider children nodes (assuming binary tree for simplicity)
        # Left child: 2 * node_index + 1
        # Right child: 2 * node_index + 2
        left_child_index = 2 * node_index + 1
        right_child_index = 2 * node_index + 2

        # Check if children exist in the scores array (i.e., not out of bounds for the tree structure)
        if left_child_index < len(scores):
            value = minimax(left_child_index, depth + 1, False, scores, tree_depth)
            best_value = max(best_value, value)
        if right_child_index < len(scores):
            value = minimax(right_child_index, depth + 1, False, scores, tree_depth)
            best_value = max(best_value, value)

        # If no children were processed (e.g., this is effectively a leaf in a sparse tree representation),
        # and it's not the base case depth, it means there are no further moves from this node.
        # In a typical game tree, you wouldn't recurse if there are no children, but with this
        # flat array representation, we need to handle it.
        if best_value == -float('inf') and depth < tree_depth:
            return scores[node_index] # If no children, treat this node's score as its value

        return best_value
    # If it's a minimizing player's turn
    else:
        best_value = float('inf') # Initialize with positive infinity
        left_child_index = 2 * node_index + 1
        right_child_index = 2 * node_index + 2

        if left_child_index < len(scores):
            value = minimax(left_child_index, depth + 1, True, scores, tree_depth)
            best_value = min(best_value, value)
        if right_child_index < len(scores):
            value = minimax(right_child_index, depth + 1, True, scores, tree_depth)
            best_value = min(best_value, value)

        if best_value == float('inf') and depth < tree_depth:
            return scores[node_index]

        return best_value

In [10]:
# Example Usage:
# Let's represent a simple game tree using an array for terminal node scores.
# The tree depth defines how many levels deep we want to look.
# Assume a binary tree structure where for a node at index 'i', its children are at '2i+1' and '2i+2'.
# The 'scores' array contains the utility values for the terminal nodes.

# Example 1: A simple game tree
# This array represents the leaf node scores of a hypothetical game tree.
# For a tree of depth 3, there would be 2^3 = 8 potential leaf nodes.
# We will fill up to 8 scores for simplicity.
# Tree structure (simplified visual, not actual array indices):
#       (Max)
#      /     \
#   (Min)    (Min)
#  /   \    /   \
# (Max)(Max)(Max)(Max)
# /\ /\ /\ /\ /\ /\ /\ /\
# 3  5 2  9 12 5 23 21 (leaf scores)

scores_ex1 = [3, 5, 2, 9, 12, 5, 23, 21]
tree_depth_ex1 = 3 # This is the depth where leaf nodes are

# Call minimax starting from the root (index 0), at depth 0, as maximizing player
optimal_value_ex1 = minimax(0, 0, True, scores_ex1, tree_depth_ex1)
print(f"Example 1: The optimal value for the Maximizing player is: {optimal_value_ex1}")


# Example 2: Another simple game tree (different scores and depth)
# Scores for a tree with a different set of terminal values.
scores_ex2 = [3, 21, 12, 7, 10, 8, 1, 9]
tree_depth_ex2 = 3

optimal_value_ex2 = minimax(0, 0, True, scores_ex2, tree_depth_ex2)
print(f"Example 2: The optimal value for the Maximizing player is: {optimal_value_ex2}")


# Example 3: A tree with fewer leaf nodes than max possible depth implies some branches end earlier.
# In this scenario, we assume if a node doesn't have children up to 'tree_depth', its own score is used.
# For simplicity of this example, we'll keep `tree_depth` to correspond to the number of levels until leaves.
# However, a more robust minimax would need to handle non-uniform tree depths more explicitly.
scores_ex3 = [50, 10, 40, 100]
tree_depth_ex3 = 2 # This means there are 2 levels of decisions before terminal nodes

optimal_value_ex3 = minimax(0, 0, True, scores_ex3, tree_depth_ex3)
print(f"Example 3: The optimal value for the Maximizing player is: {optimal_value_ex3}")

Example 1: The optimal value for the Maximizing player is: 12
Example 2: The optimal value for the Maximizing player is: 9
Example 3: The optimal value for the Maximizing player is: 100
